In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
from os.path import expanduser
## actions required!!!!!!!!!!!!!!!!!!!! change your folder path 
path_repo = expanduser("~/Documents/G3_2/regime-identification"); sys.path.append(path_repo)
path_file = expanduser("~/data/G3_2/regime-identification/simulation")
path = {}
for folder in ["data", "estimation"]:
    path[folder] = f"{path_file}/{folder}"

In [2]:
from regime.simulation_helper import *
from regime.jump import *

In [3]:
key_data = "3-state-daily"
n_s = 2000
key_feat = "HMM"
key_model = "HMM10init"

In [4]:
Zs = load_file(path, "data", key_data, "Zs", n_s)
Zs.shape

(1024, 2000)

In [5]:
proba_ = load_arr_distributed(path, "estimation", key_data, "proba", n_s, key_feat, key_model, 1, 32)
proba_.shape

(32, 2000, 3)

In [6]:
labels_ = load_arr_distributed(path, "estimation", key_data, "labels", n_s, key_feat, key_model, 1, 32)
labels_.shape

(32, 2000)

In [7]:
scorer_batch(accuracy_score, Zs[:32], labels_)

array([0.827 , 0.9055, 0.9115, 0.6865, 0.9725, 0.966 , 0.9455, 0.5245,
       0.871 , 0.8425, 0.926 , 0.9525, 0.9025, 0.8575, 0.9455, 0.766 ,
       0.9055, 0.9495, 0.939 , 0.9535, 0.8325, 0.8025, 0.94  , 0.9845,
       0.8165, 0.9275, 0.97  , 0.9   , 0.951 , 0.96  , 0.979 , 0.93  ])

In [18]:
proba_.shape

(32, 2000, 3)

In [8]:
labels_new = proba_.argmax(axis=-1)

In [9]:
scorer_batch(accuracy_score, Zs[:32], labels_new)

array([0.949 , 0.9365, 0.9235, 0.686 , 0.965 , 0.9735, 0.9365, 0.5065,
       0.871 , 0.87  , 0.933 , 0.952 , 0.9745, 0.8465, 0.9445, 0.773 ,
       0.902 , 0.9445, 0.9435, 0.956 , 0.8385, 0.7975, 0.934 , 0.988 ,
       0.8115, 0.9075, 0.9705, 0.924 , 0.95  , 0.963 , 0.978 , 0.9295])

In [25]:
labels_[2]

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [26]:
labels_new[2]

array([2, 2, 2, ..., 0, 0, 0])

In [24]:
proba_[2]

array([[0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.35558647e-04, 2.17438961e-05, 9.99842697e-01],
       [3.01925104e-04, 5.00791650e-05, 9.99647996e-01],
       ...,
       [9.58117640e-01, 3.45582285e-02, 7.32413192e-03],
       [9.49076033e-01, 4.36123471e-02, 7.31161970e-03],
       [9.43168438e-01, 4.69036872e-02, 9.92787502e-03]])

In [33]:
from numpy.random import RandomState
random_state = RandomState(0)
model = GaussianHMM_model(3, n_init=10, init='k-means++', random_state=random_state, n_iter = 500, tol = 1e-4, min_covar=1e-6, covars_prior=1e-6,
                                    transmat_prior = 1.+1e-5)#

In [36]:
Xs = load_file(path, "data", key_data, "Xs", n_s, "HMM")
X = Xs[2]

In [37]:
model.fit(X)

GaussianHMM_model(n_components=3,
                  random_state=RandomState(MT19937) at 0x13F843D40)

In [39]:
model.labels_

array([1, 1, 1, ..., 2, 2, 2], dtype=int32)

In [40]:
model.proba_

array([[3.02915422e-257, 1.00000000e+000, 8.51082384e-240],
       [2.17363477e-005, 9.99842573e-001, 1.35690518e-004],
       [5.00615115e-005, 9.99647720e-001, 3.02218462e-004],
       ...,
       [3.45417301e-002, 7.32874607e-003, 9.58129524e-001],
       [4.35904219e-002, 7.31622917e-003, 9.49093349e-001],
       [4.68803496e-002, 9.93422257e-003, 9.43185428e-001]])

In [41]:
labels_ = model.labels_
proba_ = model.proba_

In [42]:
labels_

array([1, 1, 1, ..., 2, 2, 2], dtype=int32)

In [43]:
proba_

array([[3.02915422e-257, 1.00000000e+000, 8.51082384e-240],
       [2.17363477e-005, 9.99842573e-001, 1.35690518e-004],
       [5.00615115e-005, 9.99647720e-001, 3.02218462e-004],
       ...,
       [3.45417301e-002, 7.32874607e-003, 9.58129524e-001],
       [4.35904219e-002, 7.31622917e-003, 9.49093349e-001],
       [4.68803496e-002, 9.93422257e-003, 9.43185428e-001]])

In [45]:
Z=Zs[2]

In [47]:
res= align_estimation_results_batch(Z[np.newaxis, ...], {"labels_": labels_[np.newaxis, ...], "proba_": proba_[np.newaxis, ...]})

In [70]:
def generate_all_perms_as_arr(n_c):
    return np.array(list(permutations(range(n_c))))

def permute_labels(labels_arr, all_perms):
    """
    return the labels under every permutation. new axis to the last
    """
    n_c = len(all_perms[0])
    labels_all_perms = np.zeros(labels_arr.shape + (len(all_perms),), dtype=np.int32)
    labels_all_perms[..., 0] = labels_arr
    for i_perm, perm in enumerate(all_perms[1:]):
        labels_permuted = labels_all_perms[..., i_perm+1]
        for i_cluster in range(n_c): # permute
            labels_permuted[labels_arr==perm[i_cluster]] = i_cluster# labels_permuted[labels_==i]
    return labels_all_perms


def align_estimation_results_batch(Zs_true, res):
    """
    align a batch of estimation results with the true labels, i.e. find the optimal permutation for each sample.
    results include labels_, proba_, means_, covars_, transmat_.
    """
    n_c, n_t = len(np.unique(Zs_true)), len(Zs_true)
    # all the perms
    all_perms = generate_all_perms_as_arr(n_c) 
    # all the possible perms of labels
    labels_all_perms = permute_labels(res["labels_"], all_perms)
    # score accuracy for each perm
    acc_all_perms = scorer_batch(accuracy_score, Zs_true, labels_all_perms, has_params=True) # of shape (n_t, n_p)
    # best perm for each trial 
    best_perm_idx = acc_all_perms.argmax(-1) # shape (n_t,)
    best_perm = all_perms[best_perm_idx] # (n_t, n_c)
    # take the corresponding perm for labels
    res["labels_"] = np.take_along_axis(labels_all_perms, best_perm_idx[:, np.newaxis, np.newaxis], axis=-1).squeeze(axis=-1)
    res["proba_"] = np.take_along_axis(res["proba_"], best_perm[:, np.newaxis, :], -1)
    # res["means_"] = np.take_along_axis(res["means_"], best_perm, -1)
    # res["covars_"] = np.take_along_axis(res["covars_"], best_perm, -1)
    # res["transmat_"] = np.take_along_axis(res["transmat_"][np.arange(n_t)[:, np.newaxis], best_perm], best_perm[:, np.newaxis, :], -1)
    return res

In [71]:
Zs_true=Z[np.newaxis, ...]
res={"labels_": labels_[np.newaxis, ...], "proba_": proba_[np.newaxis, ...]}

In [72]:
res = align_estimation_results_batch(Zs_true, res)

In [73]:
res['labels_']

array([[0, 0, 0, ..., 1, 1, 1]], dtype=int32)

In [74]:
res['proba_']

array([[[1.00000000e+000, 8.51082384e-240, 3.02915422e-257],
        [9.99842573e-001, 1.35690518e-004, 2.17363477e-005],
        [9.99647720e-001, 3.02218462e-004, 5.00615115e-005],
        ...,
        [7.32874607e-003, 9.58129524e-001, 3.45417301e-002],
        [7.31622917e-003, 9.49093349e-001, 4.35904219e-002],
        [9.93422257e-003, 9.43185428e-001, 4.68803496e-002]]])

In [52]:
n_c, n_t = len(np.unique(Zs_true)), len(Zs_true)
# all the perms
all_perms = generate_all_perms_as_arr(n_c) 
# all the possible perms of labels
labels_all_perms = permute_labels(res["labels_"], all_perms)
# score accuracy for each perm
acc_all_perms = scorer_batch(accuracy_score, Zs_true, labels_all_perms, has_params=True) # of shape (n_t, n_p)
# best perm for each trial 
best_perm_idx = acc_all_perms.argmax(-1) # shape (n_t,)
best_perm = all_perms[best_perm_idx]

In [65]:
labels_new = np.take_along_axis(labels_all_perms, best_perm_idx[:, np.newaxis, np.newaxis], axis=-1).squeeze(axis=-1)
proba_new = np.take_along_axis(res["proba_"], best_perm[:, np.newaxis, :], -1)

In [62]:
acc_all_perms

array([[0.045 , 0.298 , 0.4175, 0.0435, 0.9115, 0.2845]])

In [63]:
best_perm_idx

array([4])

In [64]:
best_perm

array([[2, 0, 1]])

In [55]:
res['labels_'].shape

(1, 2000)

In [58]:
labels_

array([1, 1, 1, ..., 2, 2, 2], dtype=int32)

In [66]:
labels_new

array([[0, 0, 0, ..., 1, 1, 1]], dtype=int32)

In [67]:
proba_

array([[3.02915422e-257, 1.00000000e+000, 8.51082384e-240],
       [2.17363477e-005, 9.99842573e-001, 1.35690518e-004],
       [5.00615115e-005, 9.99647720e-001, 3.02218462e-004],
       ...,
       [3.45417301e-002, 7.32874607e-003, 9.58129524e-001],
       [4.35904219e-002, 7.31622917e-003, 9.49093349e-001],
       [4.68803496e-002, 9.93422257e-003, 9.43185428e-001]])

In [68]:
proba_new

array([[[8.51082384e-240, 3.02915422e-257, 1.00000000e+000],
        [1.35690518e-004, 2.17363477e-005, 9.99842573e-001],
        [3.02218462e-004, 5.00615115e-005, 9.99647720e-001],
        ...,
        [9.58129524e-001, 3.45417301e-002, 7.32874607e-003],
        [9.49093349e-001, 4.35904219e-002, 7.31622917e-003],
        [9.43185428e-001, 4.68803496e-002, 9.93422257e-003]]])

In [60]:
all_perms

array([[0, 1, 2],
       [0, 2, 1],
       [1, 0, 2],
       [1, 2, 0],
       [2, 0, 1],
       [2, 1, 0]])

In [48]:
res['labels_']

array([[0, 0, 0, ..., 1, 1, 1]], dtype=int32)

In [49]:
res['proba_']

array([[[8.51082384e-240, 3.02915422e-257, 1.00000000e+000],
        [1.35690518e-004, 2.17363477e-005, 9.99842573e-001],
        [3.02218462e-004, 5.00615115e-005, 9.99647720e-001],
        ...,
        [9.58129524e-001, 3.45417301e-002, 7.32874607e-003],
        [9.49093349e-001, 4.35904219e-002, 7.31622917e-003],
        [9.43185428e-001, 4.68803496e-002, 9.93422257e-003]]])